## A simple swarm

In [9]:
%load_ext autoreload 
%autoreload 2
import sys
sys.tracebacklimit = 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from core.agent import *
from core.world import * 
from core.map import *
import numpy as np
from core.render import * 

ModuleNotFoundError: No module named 'v2'

In [3]:
world = World(dims = (100, 100),
              resource_generator= RandomMapGenerator(),
              energy_model=EnergyModel()
              )
swarm = [Agent() for i in range(1000)]
for agent in swarm:
    world.add_agent(agent)

In [4]:
swarm = initialize_positions_randomly(world, swarm)

In [5]:
# Test movements by doing random actions
def update():
    for agent in swarm:
        choice = np.random.randint(0, 5)
        if choice != 0: 
            agent.move(choice)

    world.update()

update()

In [6]:
render_world(world, (800, 800), update_fn=update, delay_s=0)

## Verify certain things about the simulation

In [7]:
# Validate the observation space provided is sensible
obs = swarm[0].get_observation()

print(obs.nearby_agents, obs.nearby_agents.shape)

[[  0   0   0   0   0   0   0]
 [  0   0   0   0 996   0  87]
 [  0   0   0   0   0   0 529]
 [  0   0   0   0   0 332 395]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [641   0   0   0   0   0   0]] (7, 7)


In [8]:
# Validate that all swarm agents landed in different positions
positions = set()
for agent in swarm:
    pos = agent.get_position()
    positions.add((pos[0], pos[1]))

print(len(positions), len(world.get_agents()))

1000 1000
